# LSTM for Time Series Prediction

This notebook demonstrates how to use a Long Short-Term Memory (LSTM) network for time series forecasting. 
It includes data preprocessing, model training, evaluation, and visualization.

## Steps Covered:
1. Load and preprocess the dataset
2. Create the LSTM model
3. Train the model
4. Evaluate performance
5. Visualize results

---


In [21]:
import warnings  # Import necessary libraries
warnings.simplefilter(action='ignore', category=FutureWarning)
## Import libraries developed in this study
# import hydrodeepx_data as xdata  # Import necessary libraries
# import hydrodeepx_utils as xutils  # Import necessary libraries
# import hydrodeepx_interpret as xinterpret  # Import necessary libraries
# import hydrodeepx_plot as xplot  # Import necessary libraries
from hydrodata import DataforIndividual  # Import necessary libraries
## Import dependent libraries
import os, logging, pickle  # Import necessary libraries
import numpy as np  # Import necessary libraries
import pandas as pd  # Import necessary libraries
import matplotlib.pyplot as plt  # Import necessary libraries
import tensorflow as tf  # Import necessary libraries
from tensorflow.keras import layers, models, callbacks, optimizers, regularizers  # Import necessary libraries
from tensorflow.keras import backend as K  # Import necessary libraries
## Ignore all the warnings
tf.get_logger().setLevel(logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['KMP_WARNINGS'] = '0'

plt.rcParams.update(plt.rcParamsDefault)
import pastas as ps  # Import necessary libraries
import matplotlib.pyplot as plt  # Import necessary libraries
import spi  # Import necessary libraries

ps.set_log_level("ERROR")
ps.show_versions()

Python version: 3.7.15 (default, Nov 24 2022, 18:44:54) [MSC v.1916 64 bit (AMD64)]
Numpy version: 1.21.6
Scipy version: 1.7.3
Pandas version: 1.3.5
Pastas version: 0.21.0
Matplotlib version: 3.5.3


In [2]:
#evaluating index

def cal_nse(obs, sim):

    # compute numerator and denominator
    numerator   = np.nansum((obs - sim)**2)
    denominator = np.nansum((obs - np.nanmean(obs))**2)
    # compute coefficient
    return 1 - (numerator / denominator)

def R2(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred ))
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) )
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )


def calculate_rmse(observed, predicted):  # Make predictions
    mse = np.mean((observed - predicted)**2)  # Make predictions
    rmse = np.sqrt(mse)
    return rmse

In [6]:
#Data preprocessing functions.
#read, wrap and normalize the data




def get_station_data(fname):
    """
    Obtain the pandas dataframe from dataset.

    """
    dataset = pd.read_csv(fname)  # Load dataset
    dataset["date"] = pd.to_datetime(dataset['date'], dayfirst=True,errors='ignore')
    #dataset = dataset.replace(-99, np.NaN)
    dataset = dataset.set_index("date")
    dataset = dataset.dropna()
    
    # This date range is a fake date that helps better arrange the data.
    # This can be changed with orders
    dataset = dataset.loc[pd.date_range(start='1/1/1968', end='1/1/2018')]

    return dataset


def get_wrapped_data(dataset, wrap_length=12):
    """
    Wrap the data for the shape requirement of LSTM.  # Define LSTM model

    Parameters
    ----------
    dataset: the pandas dataframe obtained from the function get_station_data().
    wrap_length: the number of time steps to be considered for the LSTM layer.  # Define LSTM model

    Returns
    ----------
    data_x_dict: the input dictionary whose key is the date and value is the corresponding wrapped input matrix of each sample.
    data_y_dict: the output dictionary whose key is the date and value is the corresponding target of each sample.
    """
    data_x_dict, data_y_dict = {}, {}

    for date_i in tqdm(dataset.index, desc=f'Prep aring data with wrap length = {wrap_length}'):
        try:
            data_x = dataset.loc[pd.date_range(end=date_i,
                                               periods=wrap_length + 1,
                                               freq="d")[:-1], ["R"], ].to_numpy(dtype='float16')
            data_y = dataset.loc[pd.date_range(end=date_i,
                                               periods=wrap_length + 1,
                                               freq="d")[-1:], "y", ].to_numpy(dtype='float16')

            data_x_dict[date_i] = data_x
            data_y_dict[date_i] = data_y
        except KeyError:
            continue

    return data_x_dict, data_y_dict


def split_train_test(dataset, data_x_dict, data_y_dict, frac=0.8, random_state=100, scale=True):
    """
    Randomly split the dataset for training and testing.

    Parameters
    ----------
    dataset: the pandas dataframe obtained from the function get_station_data().
    data_x_dict: the input dictionary obtained from the function get_wrapped_data().
    data_y_dict: the output dictionary obtained from the function get_wrapped_data().
    frac: the fraction of samples to be trained.
    random_state: the random seed (default: 100).
    scale: [bool] whether scale the split dataset by the mean and std values of the training data (default: True).

    Returns
    ----------
    train_dates: the dates of the picked training data.
    test_dates: the dates of the picked testing data.
    train_x: the (scaled) inputs for training.
    train_y: the (scaled) outputs for training.
    test_x: the (scaled) inputs for testing.
    test_y: the (scaled) outputs for testing.
    scale_params: the mean and std values of the training data (available when scale is True)
    """
    train_dates = (dataset.loc[data_x_dict.keys()].sample(frac=frac, random_state=random_state).index)
    test_dates  = dataset.loc[data_x_dict.keys()].drop(train_dates).index

    train_x = np.stack([data_x_dict.get(i) for i in train_dates.to_list()])
    train_y = np.stack([data_y_dict.get(i) for i in train_dates.to_list()])
    test_x  = np.stack([data_x_dict.get(i) for i in test_dates.to_list()])
    test_y  = np.stack([data_y_dict.get(i) for i in test_dates.to_list()])

    scale_params = {"train_x_mean": 0, "train_x_std": 1, "train_y_mean": 0, "train_y_std": 1}

    if scale is False:
        return train_dates, test_dates, train_x, train_y, test_x, test_y, scale_params
    else:
        scale_params["train_x_mean"] = (dataset.loc[train_dates, ["R"]].mean().values)
        scale_params["train_x_std"]  = (dataset.loc[train_dates, ["R"]].std().values)
        scale_params["train_y_mean"] = dataset.loc[train_dates, ["y"]].mean().values
        scale_params["train_y_std"]  = dataset.loc[train_dates, ["y"]].std().values

        train_x = (train_x - scale_params["train_x_mean"][None, None, :]) / scale_params["train_x_std"][None, None, :]
        train_y = (train_y - scale_params["train_y_mean"][None, :]) / scale_params["train_y_std"][None, :]
        test_x  = (test_x - scale_params["train_x_mean"][None, None, :]) / scale_params["train_x_std"][None, None, :]
        test_y  = (test_y - scale_params["train_y_mean"][None, :]) / scale_params["train_y_std"][None, :]

        return train_dates, test_dates, train_x, train_y, test_x, test_y, scale_params

In [7]:
# data = pd.read_csv(r"C:\TsaiHejiang\Codes\GRU_GW\03\NEW\Results\Timeseries\02296500.csv",  parse_dates=['date'],  # Load dataset
#                    index_col='date', squeeze=True)
# data.head()


In [8]:
# GW = data['GW(mm)']
# prcp = data['prcp(mm/day)']
# Tmean = data['tmean(C)']
# Stream = data['flow(mm)']

In [10]:
WORKING_PATH  = r"C:\TsaiHejiang\CMZ"

####################
#   Basin set up   #
####################
STATION_ID = '40-year-daily_#8002_1' # USGS code used in the MOPEX dataset

####################
#  Hyperparameters #
####################
RANDOM_SEED   = 100        
WRAP_LENGTH   = 180        # Timestep of the LSTM model  # Define LSTM model
TRAIN_FRAC    = 0.7        # The fraction of spliting traning and testing dataset

LEARNING_RATE = 0.03
EPOCH_NUMBER  = 100

In [11]:
mopex_path = os.path.join(WORKING_PATH, f'{STATION_ID}.csv')
data_path  = os.path.join(WORKING_PATH, 'data', f'{STATION_ID}_data.pickle')
model_path = os.path.join(WORKING_PATH, 'results', 'model', f'{STATION_ID}_{RANDOM_SEED}_keras.h5')
eg_path    = os.path.join(WORKING_PATH, 'results', 'eg',    f'{STATION_ID}_{RANDOM_SEED}_eg.pickle')

In [12]:
# hydrodata = xdata.get_station_data(fname=mopex_path)
hydrodata = get_station_data(fname=mopex_path)
print(hydrodata)

                y    R
date                  
1968-01-01  14.83  7.6
1968-01-02  13.98  3.7
1968-01-03  12.79  4.7
1968-01-04  12.45  1.8
1968-01-05  11.80  0.3
...           ...  ...
2017-12-28  15.67  0.4
2017-12-29  13.96  2.5
2017-12-30  13.41  8.4
2017-12-31  19.77  3.8
2018-01-01  22.03  3.8

[18264 rows x 2 columns]


In [13]:
# fig, [ax1, ax2, ax3] = plt.subplots(nrows=3, ncols=1, sharex=True, figsize=(8, 5))  # Visualize results
# fig.tight_layout()

# ax1.plot(hydrodata['prcp'],  'tab:blue', lw=0.8)  # Visualize results
# ax2.plot(hydrodata['tmean'], 'tab:red',  lw=0.8)  # Visualize results
# ax3.plot(hydrodata['flow'],  'tab:brown', lw=0.8)  # Visualize results

# ax1.set_title(f"Station {STATION_ID}")
# ax1.set_ylabel("prcp(mm/day)")
# ax2.set_ylabel("tmean(C)")
# ax3.set_ylabel("flow(mm)")

# plt.show()

In [14]:
print(hydrodata.index)

DatetimeIndex(['1968-01-01', '1968-01-02', '1968-01-03', '1968-01-04',
               '1968-01-05', '1968-01-06', '1968-01-07', '1968-01-08',
               '1968-01-09', '1968-01-10',
               ...
               '2017-12-23', '2017-12-24', '2017-12-25', '2017-12-26',
               '2017-12-27', '2017-12-28', '2017-12-29', '2017-12-30',
               '2017-12-31', '2018-01-01'],
              dtype='datetime64[ns]', name='date', length=18264, freq=None)


In [15]:
from tqdm.notebook import tqdm  # Import necessary libraries

In [16]:
if os.path.exists(data_path):
    with open(data_path, 'rb') as f:
        data_x_dict, data_y_dict = pickle.load(f)
else:
    data_x_dict, data_y_dict = get_wrapped_data(dataset=hydrodata,  wrap_length=WRAP_LENGTH)
    with open(data_path, 'wb') as f:
        pickle.dump([data_x_dict, data_y_dict], f)


Prep aring data with wrap length = 180:   0%|          | 0/18264 [00:00<?, ?it/s]

In [17]:
split_results = split_train_test(dataset=hydrodata, 
                                       data_x_dict=data_x_dict, 
                                       data_y_dict=data_y_dict, 
                                       frac=TRAIN_FRAC, 
                                       random_state=RANDOM_SEED, 
                                       scale=True)

train_dates, test_dates, x_train, y_train, x_test, y_test, scale_params = split_results

print(f'The shape of x_train, y_train after wrapping by {WRAP_LENGTH} days are {x_train.shape}, {y_train.shape}')
print(f'The shape of x_test, y_test after wrapping by {WRAP_LENGTH} days are   {x_test.shape}, {y_test.shape}')

The shape of x_train, y_train after wrapping by 180 days are (12659, 180, 1), (12659, 1)
The shape of x_test, y_test after wrapping by 180 days are   (5425, 180, 1), (5425, 1)


In [18]:
inputs = layers.Input(x_train.shape[1:], name='input')
lstm   = layers.LSTM(units=16, name='lstm',   # Define LSTM model
                     kernel_regularizer=regularizers.l2(0.001), 
                     recurrent_regularizer=regularizers.l2(0.001))(inputs)
output = layers.Dense(units=1, name='dense', activation='linear', use_bias=False, 
                      kernel_regularizer=regularizers.l2(0.001))(lstm)

model  = models.Model(inputs, output)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 180, 1)]          0         
_________________________________________________________________
lstm (LSTM)                  (None, 16)                1152      
_________________________________________________________________
dense (Dense)                (None, 1)                 16        
Total params: 1,168
Trainable params: 1,168
Non-trainable params: 0
_________________________________________________________________


In [19]:
# from scipy import signal  # Import necessary libraries
# from tensorflow.keras import backend as K  # Import necessary libraries
# import tensorflow as tf  # Import necessary libraries
# from tensorflow.keras import layers, models, callbacks, optimizers, regularizers  # Import necessary libraries

In [22]:
es     = callbacks.EarlyStopping(monitor='val_R2', mode='max', verbose=1, patience=30, 
                                 min_delta=0.01, restore_best_weights=True)
reduce = callbacks.ReduceLROnPlateau(monitor='val_R2', factor=0.5, patience=15, verbose=1, 
                                     mode='max', min_delta=0.01, cooldown=0, min_lr=LEARNING_RATE / 100)
tnan   = callbacks.TerminateOnNaN()

model.compile(loss='mse', metrics=[R2], optimizer=optimizers.Adam(lr=LEARNING_RATE))
model.fit(x_train, y_train, epochs=EPOCH_NUMBER, batch_size=1024, validation_split=0.3,   # Train the model
          callbacks=[es, reduce, tnan])
model.save(model_path)


Train on 8861 samples, validate on 3798 samples
Epoch 1/100
8861/8861 [==============================] - 3s 363us/sample - loss: 0.6065 - R2: 0.4081 - val_loss: 0.4645 - val_R2: 0.5586
Epoch 2/100
8861/8861 [==============================] - 2s 199us/sample - loss: 0.4473 - R2: 0.5718 - val_loss: 0.4106 - val_R2: 0.6201
Epoch 3/100
8861/8861 [==============================] - 2s 215us/sample - loss: 0.4255 - R2: 0.5920 - val_loss: 0.3900 - val_R2: 0.6349
Epoch 4/100
8861/8861 [==============================] - 2s 212us/sample - loss: 0.4021 - R2: 0.6165 - val_loss: 0.3795 - val_R2: 0.6456
Epoch 5/100
8861/8861 [==============================] - 2s 201us/sample - loss: 0.3955 - R2: 0.6157 - val_loss: 0.3926 - val_R2: 0.6284
Epoch 6/100
8861/8861 [==============================] - 2s 205us/sample - loss: 0.4068 - R2: 0.6099 - val_loss: 0.3784 - val_R2: 0.6431
Epoch 7/100
8861/8861 [==============================] - 2s 202us/sample - loss: 0.4010 - R2: 0.6143 - val_loss: 0.3687 - val_R2: 

Epoch 58/100
8861/8861 [==============================] - 2s 204us/sample - loss: 0.3463 - R2: 0.6694 - val_loss: 0.3222 - val_R2: 0.6963
Epoch 59/100
8861/8861 [==============================] - 2s 217us/sample - loss: 0.3455 - R2: 0.6646 - val_loss: 0.3186 - val_R2: 0.6974
Epoch 60/100
8861/8861 [==============================] - 2s 214us/sample - loss: 0.3438 - R2: 0.6688 - val_loss: 0.3214 - val_R2: 0.7022
Epoch 61/100
8861/8861 [==============================] - 2s 214us/sample - loss: 0.3470 - R2: 0.6664 - val_loss: 0.3228 - val_R2: 0.6971
Epoch 62/100
8861/8861 [==============================] - 2s 219us/sample - loss: 0.3484 - R2: 0.6658 - val_loss: 0.3303 - val_R2: 0.6934
Epoch 63/100
8861/8861 [==============================] - 2s 229us/sample - loss: 0.3510 - R2: 0.6635 - val_loss: 0.3204 - val_R2: 0.7033
Epoch 64/100
8861/8861 [==============================] - 2s 230us/sample - loss: 0.3465 - R2: 0.6647 - val_loss: 0.3208 - val_R2: 0.6972
Epoch 65/100
8861/8861 [==========

In [ ]:
eval_model = models.load_model(model_path, custom_objects={'R2': xutils.R2})
pred_train = eval_model.predict(x_train, batch_size=1024)  # Make predictions
pred_test  = eval_model.predict(x_test, batch_size=1024)  # Make predictions

print(f"NSE for the training data: {xutils.cal_nse(y_train, pred_train):.3f}")
print(f"NSE for the testing data:  {xutils.cal_nse(y_test, pred_test):.3f}")

In [ ]:
hydrodata.loc[train_dates, ['flow_pred']] = pred_train * scale_params['train_y_std'] + scale_params['train_y_mean']
hydrodata.to_csv(r'C:\TsaiHejiang\CMZ\results\results_train_daily.csv')


In [ ]:
#Plot the train and test
hydrodata.loc[test_dates,  ['flow_pred']] = pred_test  * scale_params['train_y_std'] + scale_params['train_y_mean']
hydrodata.to_csv(r'C:\TsaiHejiang\CMZ\results\results_daily.csv')
